In [1]:
# Import Dependencies.
import pandas as pd
from citipy import citipy
import requests
from config import weather_api_key
import time
from datetime import datetime
import numpy as np

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

754

In [5]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [28]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_description = city_weather["weather"][-1]["description"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]

        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                        "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_description})
                          

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | cape town
Processing Record 2 of Set 1 | busselton
Processing Record 3 of Set 1 | tawala
Processing Record 4 of Set 1 | castro
Processing Record 5 of Set 1 | mataura
Processing Record 6 of Set 1 | punta arenas
Processing Record 7 of Set 1 | longido
Processing Record 8 of Set 1 | padang
Processing Record 9 of Set 1 | tumannyy
City not found. Skipping...
Processing Record 10 of Set 1 | ancud
Processing Record 11 of Set 1 | new norfolk
Processing Record 12 of Set 1 | forio
Processing Record 13 of Set 1 | bethel
Processing Record 14 of Set 1 | nanortalik
Processing Record 15 of Set 1 | montepuez
Processing Record 16 of Set 1 | rikitea
Processing Record 17 of Set 1 | san juan
Processing Record 18 of Set 1 | marystown
Processing Record 19 of Set 1 | talnakh
Processing Record 20 of Set 1 | grenaa
Processing Record 21 of Set 1 | cabo san lucas
Processing Record 22 of Set 1 | east london
Processing Record

In [34]:
weatherpy_df = pd.DataFrame(city_data)
weatherpy_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Cape Town,ZA,-33.9258,18.4232,55.90,82,0,8.01,clear sky
1,Busselton,AU,-33.6500,115.3333,51.26,84,0,7.67,clear sky
2,Tawala,PH,9.5542,123.7696,78.76,84,100,12.39,overcast clouds
3,Castro,BR,-24.7911,-50.0119,56.80,99,100,4.16,overcast clouds
4,Mataura,NZ,-46.1927,168.8643,45.48,94,100,2.10,light rain
...,...,...,...,...,...,...,...,...,...
690,Kroonstad,ZA,-27.6504,27.2349,45.84,50,0,4.56,clear sky
691,Port Macquarie,AU,-31.4333,152.9167,55.89,65,30,11.74,scattered clouds
692,Castro-Urdiales,ES,43.3829,-3.2204,69.62,79,91,9.42,overcast clouds
693,Rzhyshchiv,UA,49.9689,31.0463,56.08,73,47,6.69,scattered clouds


In [42]:
weatherpy_df.to_csv("Weather_Database/WeatherPy_Database.csv")